In [4]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

## Set up parameters, create isochrone grid

In [5]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F200W']
filters = ['m_jwst_F115W', 'm_jwst_F200W']
iso_dir = 'isochrones/'
metallicity = 0

# Select nbody6 simulation and snapshot, then convert to physical units and save as a cluster table which SPISEA can interface with

In [6]:
# read nbody data
path_to_data = '../../../../../../../storage/jfarias/Data/Orion/newbinaries/M3000sigma0.1/sfeff001_00/'
snapshot = Reader.read_snapshot(path_to_data, time = 1.5)

# convert snapshot values from nbody units to astrophysical, extract masses
snapshot.to_physical()
cluster_table = converter.to_spicea_table(snapshot)

print(cluster_table)

        mass        isMultiple        compMass       ...       log_a                age        
------------------- ---------- --------------------- ... ------------------ -------------------
 0.6824193596839905        0.0                    [] ...                nan  1.5194052457809448
0.04482797905802727        0.0                    [] ...                nan  1.5194052457809448
0.24105285108089447        0.0                    [] ...                nan  1.5194052457809448
0.10110508650541306        0.0                    [] ...                nan  1.5194052457809448
                ...        ...                   ... ...                ...                 ...
0.04427308216691017        1.0 [0.01122098509222269] ... 2.8329200549620057  1.5194052457809448
 0.1970241656375069        1.0 [0.06549611049251275] ... 3.3722471289467872  0.7565748221914069
 0.4568835469158771        1.0 [0.18551863025181375] ...  3.147479352833872  0.6406007790565741
 0.1151371643489553        1.0 [0.095487

# Read ages of stars, identify bracketing ages for isochrone generation
Also save mass of stars

In [7]:
masses = cluster_table['mass']
level_ages = cluster_table['age']

for i in range(len(level_ages)):
    level_ages[i] = level_ages[i] * 1e6

# convert log ages from log(Myr) to log(yr)
log_ages = np.log10(level_ages)

# print(log_ages)
print("Min log age = " + str(min(log_ages)))
print("Max log age = " + str(max(log_ages)))

# print(level_ages)
print("Min level age = " + str(min(level_ages)))
print("Max level age = " + str(max(level_ages)))

Min log age = 5.160646094513871
Max log age = 6.181673621482367
Min level age = 144759.17338916156
Max level age = 1519405.2457809448


# Identify binary stars in cluster table, save mass of companion stars

Cluster table always lists binaries last, so finding the index of the first binary suffices

In [8]:
# find index of first binary, create array of companion masses
first_binary = 0
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        first_binary = i
        break
# print(first_binary)

compMasses = []
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        compMasses.append(cluster_table['compMass'][i][0])
# print(compMasses)

# Generate isochrones using parameter and age arrays

In [9]:
increment = 3000
start = (min(level_ages) // increment) * increment
end = (max(level_ages) // increment) * increment + 2 * increment

# Create age arrays
level_age_arr = np.arange(start, end, increment)
log_age_arr = np.log10(level_age_arr)

print(log_age_arr)
print(level_age_arr)

[5.15836249 5.16731733 5.17609126 5.18469143 5.1931246  5.20139712
 5.20951501 5.21748394 5.22530928 5.23299611 5.24054925 5.24797327
 5.25527251 5.26245109 5.26951294 5.2764618  5.28330123 5.29003461
 5.29666519 5.30319606 5.30963017 5.31597035 5.32221929 5.3283796
 5.33445375 5.34044411 5.34635297 5.35218252 5.35793485 5.36361198
 5.36921586 5.37474835 5.38021124 5.38560627 5.39093511 5.39619935
 5.40140054 5.40654018 5.41161971 5.41664051 5.42160393 5.42651126
 5.43136376 5.43616265 5.44090908 5.4456042  5.45024911 5.45484486
 5.45939249 5.46389299 5.46834733 5.47275645 5.47712125 5.48144263
 5.48572143 5.48995848 5.49415459 5.49831055 5.50242712 5.50650503
 5.51054501 5.51454775 5.51851394 5.52244423 5.52633928 5.5301997
 5.53402611 5.5378191  5.54157924 5.54530712 5.54900326 5.55266822
 5.5563025  5.55990663 5.56348109 5.56702637 5.57054294 5.57403127
 5.5774918  5.58092498 5.58433122 5.58771097 5.59106461 5.59439255
 5.59769519 5.6009729  5.60422605 5.60745502 5.61066016 5.613841

In [10]:
# create isochrone grid - if this is the first time, then this is going to take an hour lmfao
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/wyz5rge/models//evolution/Baraffe15/iso/z015/iso_5.16.fits'

In [ ]:
# identify array indeces, create array for labels and colors
idx_arr = range(0, len(log_age_arr))
if len(idx_arr) > 3:
    idx_arr = idx_arr[::2]

# Plot CMD
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)
'''
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
'''
for i in range(len(log_age_arr)):
    if i == 0:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F200W')
py.ylabel('F200W')
py.gca().invert_yaxis()
py.legend()

# Use generated SPISEA isochrones to find interpolated magnitudes using the age and mass of each star

Stars outside of the mass range of isochrones are thrown out and not included

In [ ]:
import sys
sys.path.append('/scratch/wyz5rge/synthetic-hr/')
import interpolator as interpolator

level_ages_myr = level_ages / 1e6
# print(level_ages_myr)

# create array of stars
stars = np.empty(len(cluster_table), dtype=object)

# perform interpolation for each star
for i in range(len(stars)):
    # print(str(i) + " " + str(level_ages_myr[i]) + " " + str(masses[i]))
    stars[i] = interpolator.interpolate(level_ages_myr[i], masses[i], instances, log_age_arr, filters)
    
# convert luminosity values to solar luminosities
watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun

for i in range(len(stars)):
    if stars[i] is None:
        continue
    stars[i][0] = stars[i][0] * watts_to_lsun
    
# for i in range(len(stars)):
    # print(stars[i])

# Repeat interpolation on companion stars

Companion stars assumed to be the same age as primary star within system

In [ ]:
# perform interpolation on companion stars
companions = np.empty(len(compMasses), dtype=object)

for i in range(len(companions)):
    if stars[i + first_binary] is None:
        continue
    companions[i] = interpolator.interpolate(level_ages_myr[i + first_binary], compMasses[i], instances, log_age_arr, filters)

for i in range(len(companions)):
    if companions[i] is None:
        continue
    companions[i][0] = companions[i][0] * watts_to_lsun


# Superimpose primary stars and companion stars against isochrones separately

In [ ]:
# plot primaries and companions separately
fig, axes = py.subplots(figsize=(15, 10))

# plot primaries
py.subplot(1, 2, 1)
for i in range(len(log_age_arr)):
    if i == 0:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'red', label='')
    
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.title('CMD of primary stars only')
py.legend()

# plot companions
py.subplot(1, 2, 2)
for i in range(len(log_age_arr)):
    if i == 0:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'red', label='')
        
for i in range(len(companions)):
    if companions[i] is None:
        continue
    py.plot(companions[i][3] - companions[i][4], companions[i][4], marker='o', markersize=1, color='k')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.title('CMD of companion stars only, post-filtering')
py.legend()

# Convert binary stars into unresolved binaries
## Also compute flux ratios of stars and generate histogram
Magnitudes of primary and secondary are converted into flux, which is used to find combined magnitude

In [ ]:
# combine magnitudes on binary stars
unresolved_binaries = np.empty(len(companions), dtype=object)

for i in range(len(companions)):
    if stars[i + first_binary] is None or companions[i] is None:
        continue
    
    flux11 = np.power(10, stars[i + first_binary][3] / -2.5) * 3.93e-10
    flux21 = np.power(10, companions[i][3] / -2.5) * 3.93e-10
    mag1 = -2.5 * np.log10((flux11 + flux21) / 3.93e-10)
    
    flux12 = np.power(10, stars[i + first_binary][4] / -2.5) * 5.74e-11
    flux22 = np.power(10, companions[i][4] / -2.5) * 5.74e-11
    mag2 = -2.5 * np.log10((flux12 + flux22) / 5.744e-11)
    
    unresolved_binaries[i] = [mag1, mag2]

# Generate CMD using star array, plus unresolved binaries
Compare to CMD generated by simply multiplying each star's flux by 2

In [ ]:
fig, axes = py.subplots(figsize=(15, 10))

# plot CMD
py.subplot(1, 2, 1)
for i in range(len(log_age_arr)):
    if i == 0:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'red', label='')

# plot single stars first
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')

# plot unresolve binaries
for i in range(len(unresolved_binaries)):
    if unresolved_binaries[i] is None:
        continue
    py.plot(unresolved_binaries[i][0] - unresolved_binaries[i][1], unresolved_binaries[i][1], marker='o', markersize=1, color='green')

py.xlabel('F115W - F200W')
py.ylabel('F200W')
py.gca().invert_yaxis()
py.title('CMD of stars with unresolved binaries')
py.legend()


py.subplot(1, 2, 2)

for i in range(len(log_age_arr)):
    if i == 0:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points[filters[0]] - instances[i].points[filters[1]], 
           instances[i].points[filters[1]], 'red', label='')

# plot single stars first
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')

# plot unresolve binaries
for i in range(len(unresolved_binaries)):
    if unresolved_binaries[i] is None:
        continue
    py.plot(stars[first_binary + i][3] - stars[first_binary + i][4], 
            unresolved_binaries[i][1], marker='o', markersize=1, color='green')
    
idx_0p5 = np.where( abs(instances[1].points['mass'] - 0.1) == min(abs(instances[1].points['mass'] - 0.1)) )[0]
py.plot(instances[1].points[filters[0]][idx_0p5] - instances[1].points[filters[1]][idx_0p5],
       instances[1].points[filters[1]][idx_0p5], ms=15, marker='*', color='r', label='0.1 $M_\odot$')

idx_0p8 = np.where( abs(instances[1].points['mass'] - 0.3) == min(abs(instances[1].points['mass'] - 0.3)) )[0]
py.plot(instances[1].points[filters[0]][idx_0p8] - instances[1].points[filters[1]][idx_0p8],
       instances[1].points[filters[1]][idx_0p8], ms=15, marker='*', color='b', label='0.3 $M_\odot$')

py.xlabel('F115W - F200W')
py.ylabel('F200W')
py.gca().invert_yaxis()
py.title('CMD with unresolved binary (primary color only)')
py.legend()

In [ ]:
# Calculate spread in 0.1-0.3 Msun range using simple standard deviation calculation
mags = []

count = 0
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    if masses[i] > 0.1 and masses[i] < 0.3:
            mags.append(stars[i][4])
            count = count + 1

for i in range(len(unresolved_binaries)):
    if unresolved_binaries[i] is None:
        continue
    if masses[first_binary + i] > 0.1 and masses[first_binary + i] < 0.3:
        mags.append(unresolved_binaries[i][1])
        count = count + 1

print(count)

std = np.std(mags)
print("Standard deviation:" + str(std))

In [ ]:
temperature_bins = np.arange(3000, 3700, 100)
standard_deviations = {temp: [] for temp in temperature_bins}

# Populate temperature bins and calculate standard deviations
for temp in temperature_bins:
    magnitudes = []
    for i in range(0, first_binary):
        if stars[i] is None:
            continue
        if temp <= stars[i][1] < temp + 100:
            magnitudes.append(stars[i][4])
    for i in range(len(unresolved_binaries)):
        if unresolved_binaries[i] is None or stars[i + first_binary] is None:
            continue
        if temp <= stars[i + first_binary][1] < temp + 100:
            magnitudes.append(unresolved_binaries[i][1])
    
    standard_deviation = np.std(magnitudes)
    standard_deviations[temp] = standard_deviation

# Print standard deviations for each temperature bin
for temp, std_dev in standard_deviations.items():
    print(f"Temperature: {temp} K, Standard Deviation: {std_dev}")
            